In [1]:
# data processing
import numpy as np
import pandas as pd

# GIS calculation
import geopandas as gpd

# network model
import networkx as nx

In [2]:
# import lion nodes. I extracted a subset of the raw LION dataset within the boundary of Manhattan.
gdf_lion_nodes = gpd.read_file('../../data/raw/lion/nodes_manhattan.geojson')

# import lion edges (streets). I extracted a subset of the raw LION dataset within the boundary of Manhattan.
# I will upload this edge geojson file into the shared google drive folder
gdf_lion_edges = gpd.read_file('../../data/raw/lion/edges_manhattan.geojson')

In [3]:
# import the neighborhood boundary (from https://data.cityofnewyork.us/City-Government/Neighborhoods-Boundries/j2bc-fus8)
gdf_neighborhood = gpd.read_file('../../data/raw/boundary/Neighborhoods Boundries.geojson')

In [4]:
# extract a boundary of east village
gdf_east_village = gdf_neighborhood.loc[gdf_neighborhood.loc[:,'ntaname'] == 'East Village']

# change a coordinate system
gdf_east_village = gdf_east_village.to_crs(2263)

In [5]:
# filter edges with east village boundary
gdf_lion_edges_ev = gpd.sjoin(gdf_lion_edges,gdf_east_village.loc[:,['ntaname','geometry']],
                                how='inner',
                                op='intersects').drop(['index_right','ntaname'], axis=1)

c:\Users\seung\anaconda3\envs\teamgigo\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [6]:
# create a graph, with length of each edge
# because I assumed that the street network is undirected single graph. 
# so some edges will be removed if there are multiple streets between two nodes.
G_lion_ev = nx.from_pandas_edgelist(gdf_lion_edges_ev, 'NodeIDFrom', 'NodeIDTo', edge_attr=['OBJECTID','SHAPE_Length'], create_using=nx.Graph())

In [7]:
# https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html#networkx.algorithms.centrality.edge_betweenness_centrality
betweenness_centrality = nx.edge_betweenness_centrality(G_lion_ev, weight='SHAPE_Length')

In [8]:
# add betweeness centrality as a property of the edges
for k, v in betweenness_centrality.items():
    node_id_from = k[0]
    node_id_to = k[1]
    # 각 엣지에 edge_id 속성 추가
    G_lion_ev[node_id_from][node_id_to]['betweeness'] = v

In [9]:
# convert the edge info as a dataframe. 
df_edge_info = pd.DataFrame([value for _,_,value in list(G_lion_ev.edges(data=True))])

In [10]:
df_edge_info.loc[:,['OBJECTID','betweeness']]

,OBJECTID,betweeness
0,92330,0.037413
1,92451,0.068470
2,99332,0.068993
3,99490,0.050417
4,94065,0.115926
...,...,...
339,121404,0.000058
340,121413,0.007605
341,121410,0.015093
342,121407,0.000087


In [11]:
# merge 
gdf_lion_edges_ev.merge(df_edge_info.loc[:,['OBJECTID','betweeness']], how='inner', on='OBJECTID')

,OBJECTID,Street,SAFStreetName,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,...,ToLeft,FromRight,ToRight,Join_ID,L_PD_Service_Area,R_PD_Service_Area,TRUCK_ROUTE_TYPE,SHAPE_Length,geometry,betweeness
0,92330,EAST 10 STREET,,0,U,,B,,W,DOT,...,291,246,298,1137001000000,,,,697.101964,"MULTILINESTRING ((988564.952 204786.940, 98917...",0.037413
1,92401,1 AVENUE,,0,U,,B,,W,DOT,...,145,134,148,1248001000000,,,2,244.717371,"MULTILINESTRING ((988326.051 204358.591, 98844...",0.082779
2,92412,ASTOR PLACE,ASTOR PLACE,0,U,,B,,W,DOT,...,101,0,0,11182001000000A,,,,249.194441,"MULTILINESTRING ((986820.543 205191.191, 98704...",0.007750
3,92428,3 AVENUE,,0,U,,B,,T,DOT,...,124,107,123,1437001000000,,,2,286.917145,"MULTILINESTRING ((987663.540 206184.105, 98780...",0.029838
4,92444,EAST 4 STREET,MERCHANTS HOUSE MUSEUM,0,U,,B,,W,DOT,...,0,0,0,11284501000000N,,,,524.020965,"MULTILINESTRING ((986149.530 204458.282, 98659...",0.007605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,127784,14 STREET/CANARSIE LINE,,1,U,,B,,,,...,0,0,0,1002606020000,,,,259.217862,"MULTILINESTRING ((989814.967 205331.420, 99004...",0.022756
340,127833,14 STREET/CANARSIE LINE,,1,U,,B,,,,...,0,0,0,1002606020000,,,,336.290842,"MULTILINESTRING ((987048.269 206851.147, 98734...",0.008475
341,127834,14 STREET/CANARSIE LINE,,1,U,,B,,,,...,0,0,0,1002606020000,,,,18.981875,"MULTILINESTRING ((987343.056 206689.306, 98735...",0.005515
342,127835,14 STREET/CANARSIE LINE,,1,U,,B,,,,...,0,0,0,1002606020000,,,,382.457297,"MULTILINESTRING ((987359.695 206680.171, 98769...",0.017067
